In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
from art.vllm import get_llm
import torch
import vllm

llm = await get_llm(
    vllm.AsyncEngineArgs(
        model="Qwen/Qwen3-32B",
        # pipeline_parallel_size=1,
        tensor_parallel_size=torch.cuda.device_count(),
        # data_parallel_size=1,
        enforce_eager=True,
        generation_config="vllm",
    ),
)

In [ ]:
import art
from art.vllm import openai_server_task

sleep_task = await openai_server_task(
    engine=llm,
    config=art.dev.OpenAIServerConfig(
        log_file="./vllm.log",
        server_args=art.dev.ServerArgs(
            api_key="default",
            return_tokens_as_token_ids=True,
            enable_auto_tool_choice=True,
            tool_call_parser="hermes",
        ),
        engine_args=art.dev.EngineArgs(
            model=llm.vllm_config.model_config.model,
            disable_log_requests=True,
            generation_config="vllm",
        ),
    ),
)

In [ ]:
await llm.sleep()

In [6]:
await llm.wake_up()

In [ ]:
from art.vllm import run_on_workers
import asyncio
import os
import signal

pids = await run_on_workers(llm, lambda: os.getpid())
pids

In [8]:
from art.vllm import get_worker


def sleep() -> None:
    worker = get_worker()
    worker.sleep()
    os.kill(os.getpid(), signal.SIGSTOP)
    worker.wake_up()


sleep_task = asyncio.create_task(run_on_workers(llm, sleep))

In [ ]:
for pid in pids:
    os.kill(pid, signal.SIGCONT)
await sleep_task

In [11]:
llm.shutdown()